To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 8192  # Increased for your longer MDA texts
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

*   List item

> Add blockquote


*   List item



In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,  # Increased for small dataset
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Matched to r
    lora_dropout = 0.1,  # Added dropout for small dataset
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.8.1 patched 16 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [11]:
from datasets import load_dataset, Dataset
import json
dataset_path = "/content/drive/MyDrive/stock_prediction_data_colab.jsonl"  # Update this path
with open(dataset_path, 'r') as f:
    data = [json.loads(line) for line in f]
dataset = Dataset.from_list(data)
dataset = dataset.map(formatting_prompts_func, batched=True)

print(f"📊 Dataset loaded: {len(dataset)} examples")
print(f"📝 Sample text preview:")
print(dataset[0]["text"][:500] + "...")


Map:   0%|          | 0/35 [00:00<?, ? examples/s]

📊 Dataset loaded: 35 examples
📝 Sample text preview:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are a financial analyst expert at predicting stock performance based on Management Discussion and Analysis (MDA) sections from annual reports. Analyze the provided MDA text and predict the stock's performance for the next 12 months.<|eot_id|><|start_header_id|>user<|end_header_id|>

Analyze the following Management Discussion and Analysis (MDA) from BAJAJFINSV's 2016 a...


In [9]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [10]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [9]:
dataset[5]["conversations"]

[{'content': 'How do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?',
  'role': 'user'},
 {'content': 'Astronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.',
  'role': 'assistant'}]

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [10]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAstronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.<|

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [12]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False,
    args = SFTConfig(
        per_device_train_batch_size = 1,  # Small for long sequences
        gradient_accumulation_steps = 8,  # Effective batch size = 8
        warmup_steps = 5,
        num_train_epochs = 2,  # More epochs for small dataset
        learning_rate = 1e-4,  # Lower LR for stability
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",  # Better for small datasets
        seed = 3407,
        output_dir = "/content/drive/MyDrive/stock_model_outputs",
        save_steps = 10,  # Save frequently
        eval_steps = 10,
        save_total_limit = 3,
        report_to = "none",
        dataloader_pin_memory = False,
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/35 [00:00<?, ? examples/s]

In [11]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

**We** also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [13]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/35 [00:00<?, ? examples/s]

In [12]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

We verify masking is actually done:

In [14]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a financial analyst expert at predicting stock performance based on Management Discussion and Analysis (MDA) sections from annual reports. Analyze the provided MDA text and predict the stock's performance for the next 12 months.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nAnalyze the following Management Discussion and Analysis (MDA) from BAJAJFINSV's 2025 annual report and predict the stock performance for the next 12 months.\n\nCompany: BAJAJFINSV\nReport Year: 2025\n\nMDA Text:\n# The management discussion and analysis section was from page 53 to page 80. # MANAGEMENT DISCUSSION AND ANALYSIS ## Complete MD&A Content Bajaj Finserv Ltd. (‘Bajaj Finserv’, ‘BFS’ or ‘the Company’) is an unregistered core investment company (CIC) under Reserve Bank Regulations. It is the holding company for various financial services businesse

In [15]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [16]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.203 GB of memory reserved.


We can see the System and Instruction prompts are successfully masked!

In [17]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.203 GB of memory reserved.


In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 35 | Num Epochs = 2 | Total steps = 10
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 22,544,384 of 1,258,358,784 (1.79% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.582100
2,2.023700
3,1.799300
4,1.648200
5,0.672300
6,0.880900
7,0.639000
8,0.467200
9,0.478600
10,0.611000


In [19]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

260.4211 seconds used for training.
4.34 minutes used for training.
Peak reserved memory = 3.85 GB.
Peak reserved memory for training = 2.647 GB.
Peak reserved memory % of max memory = 26.118 %.
Peak reserved memory for training % of max memory = 17.957 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [20]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": "You are a financial analyst expert at predicting stock performance based on Management Discussion and Analysis (MDA) sections from annual reports. Analyze the provided MDA text and predict the stock's performance for the next 12 months."},
    {"role": "user", "content": "Analyze the following Management Discussion and Analysis (MDA) from BAJAJFINSV's 2025 annual report and predict the stock performance for the next 12 months.\n\nCompany: BAJAJFINSV\nReport Year: 2025\n\nMDA Text:\n# The management discussion and analysis section was from page 53 to page 80. # MANAGEMENT DISCUSSION AND ANALYSIS ## Complete MD&A Content Bajaj Finserv Ltd. (\u2018Bajaj Finserv\u2019, \u2018BFS\u2019 or \u2018the Company\u2019) is an unregistered core investment company (CIC) under Reserve Bank Regulations. It is the holding company for various financial services businesses under the Bajaj group. With a vision to be a diversified financial services group with a pan-India presence, it offers lifecycle financial solutions for its various customers. The mapping of financial lifecycle needs of the customers and offerings by BFS group along with the role BFS plays, beyond just a holding Company, has been detailed in the Chairman\u2019s letter forming part of this annual report. As a core investment company (CIC) under the regulations of the RBI, Bajaj Finserv is required to invest at least 90% of its net assets in group companies, of which at least 60% should be in the form of equity investments. Investments outside the group can only be made in specified short-term securities like money market instruments. As Bajaj Finserv does not access public funds, it does not require registration with the Reserve Bank of India. Bajaj Finserv has invested in renewable energy through 138 windmills in Maharashtra with an aggregate installed capacity of 65.2 MW. It generates more renewable energy than the energy BFS group consumes in aggregate annually. This helps BFS group fulfill its environmental obligations. As required by regulation, the standalone and consolidated financial results of Bajaj Finserv are compliant with Indian Accounting Standards (Ind AS). The insurance companies are not covered under Ind AS. They prepare Ind AS financials only for the purpose of consolidation with Bajaj Finserv. Accordingly, the financials in the sections on Bajaj Allianz General Insurance Company Ltd. (BAGIC) and Bajaj Allianz Life Insurance Company Ltd. (BALIC) in this chapter are as per Indian Generally Accepted Accounting Principles (Indian GAAP) and regulations laid down by the Insurance Regulatory and Development Authority of India (IRDAI). Performance of subsidiary companies is detailed in subsequent pages.Given below are highlights during the year. BFS: Highlights for FY2025 \u2022 BFS has signed Share Purchase Agreements (SPAs) to acquire 26% interest owned by Allianz SE in Bajaj Allianz General Insurance Company (BAGIC) and Bajaj Allianz Life Insurance Company (BALIC). This will increase the Bajaj group\u2019s ownership in BAGIC and BALIC to 100%. \u2022 Bajaj Finance Ltd.\u2019s (BFL) customer franchise surpassed a milestone of 100 million and consolidated assets under management (AUM) crossed I 400,000 crore. BFL is now the largest NBFC in India in terms of AUM and PAT. \u2022 Bajaj Housing Finance Ltd. (BHFL) successfully concluded its Initial Public Offer (IPO) of I 6,560 crore. \u2022 BAGIC continued to be the 3rd largest general insurer, measured by gross direct premium, ahead of three public sector insurers of significantly longer vintage. \u2022 BALIC maintained 6th rank in individual rated new business premium among top 10 private players. BALIC\u2019s AUM crossed I 120,000 crore. \u2022 Bajaj Finserv Health entered hospitalisation claims management with the successful acquisition of 100% stake in Vidal Healthcare Services Pvt. Ltd. \u2022 BFS AMC recorded AUM of I 20,365 crore in its first full year of operation. It was the fastest in the industry to achieve this. BFS K 133,822 crore ALL TIME HIGH CONSOLIDATED REVENUEK 8,872 crore ALL TIME HIGH CONSOLIDATED PROFIT Macroeconomic Overview According to the latest update of the IMF\u2019s World Economic Outlook (April 2025), global growth is projected at 2.8% in CY2025 and 3.0% in CY2026. This marks a sharp downward revision primarily driven by rapid escalation of trade tensions and persistently high levels of policy uncertainty, both of which are expected to significantly dampen global economic activity. Meanwhile, the global headline inflation is expected to ease to 4.3% in CY2025 and then to 3.6% in CY2026. In a scenario where global growth was forecasted at 2.8% for CY2025, the IMF estimated India\u2019s real GDP growth in 2024-25 (FY2025) at 6.5% \u2014 which was significantly higher than all major countries including China. This was the case in the previous year; and the IMF expects it to remain the same in the next couple of years as well. According to the Second Advance Estimates released by the National Statistical Office (NSO) on 28 February 2025, real GDP for FY2025 has been pegged at 6.5%; analogously, the growth of real gross value added (GVA) has been estimated at 6.4%. Though these growth rates are lower than those of the previous financial year \u2014 GDP growth at 6.5% versus 9.2% in FY2024, and GVA growth at 6.4% versus 8.6% in FY2024 \u2014 these are still creditable rates of growth. The quarterly growth rates have also been commendable. Regarding real GDP, the first quarter (April-June 2024) saw a growth of 6.5% versus the same quarter of the previous year; Q2 (July-September) witnessed a sharp dip in growth to 5.6%; and then in Q3 (October-December), growth again recovered to 6.2%. The quarterly GVA growth rates were quite similar: 6.5% in Q1, followed by 5.8% in Q2 and then 6.2% in Q3. In terms of real GVA across the three broad sectors: \u2022 The primary sector (agriculture, livestock, forestry and fishing and mining and quarrying) has grown by 4.4% over the first three quarters of FY2025. \u2022 The secondary sector (manufacturing, electricity, gas, water supply and other utility services and construction) has grown by 5.8%. \u2022 The tertiary sector (trade, hotels, transport, communication and services related to broadcasting, financial, real estate and professional services and public administration, defence and other services) has grown by 7.3%. The major drivers of growth have been the tertiary sector and construction, though these have grown at marginally lower levels compared to the previous year. The disappointment has been manufacturing, which has grown at 4.3% in the first three quarters of FY2025 versus 12.3% in the corresponding previous year. Good news has been the continued moderation of price inflation. Starting from April 2024 for every month right up to February 2025, inflation based upon the Wholesale Price Index (WPI) has remained well under 4%. Indeed, for 10 of these 11 months, it has been at under 3%. Over the same period, inflation of the Consumer Price Index for Industrial Workers (CPI-IW) stayed between 2% and 4%, except for two months (September and October 2024) when it marginally exceeded 4%. Similarly, inflation based on the Consumer Price Index for Agricultural Labourers (CPI-AL) began with a high of just above 7% in April 2024 but then steadily fell over the period to a tad over 4% in February 2025. Thus, it would seem that inflation is under control. Therefore, India has enjoyed creditable GDP and GVA growth without inflation raising its ugly head.With respect to the current account deficit (CAD), India\u2019s CAD for Q3 of FY2025 (October-December 2024) stood at $11.5 billion, or 1.1% of GDP. Though this was slightly higher than the $10.4 billion (1.1% of GDP) deficit in the same period the previous year, it has moderated from the $16.7 billion (1.8% of GDP) deficit in Q2 of FY2025. For an economy growing at no less than 6.5% real and with no near-term inflation risks, this is not a CAD that one should worry about. The RBI has estimated real GDP growth for FY2025 at 6.5%. It has projected growth at the same percentage rate for FY2026; and has forecast CPI inflation at 4%. With no imminent fear of inflation, the RBI\u2019s Monetary Policy Committee, in its first meeting of FY2026, unanimously decided to reduce the policy repo rate by 25 basis points, bringing it down to 6 per cent with immediate effect. Therefore, as it stands, the Indian economy seems to be in a good place. At 6.5%, it continues to clock the highest real GDP growth rate among all major countries in the world, including China. Both wholesale and consumer price inflation are under control. The CAD is reasonable given the size and growth of the economy. And the central bank has been confident enough to reduce the repo rate and inject further liquidity into the system. A key concern remains the recent escalation of US tariffs and policymakers will have to watch this very carefully in the coming months. Even so, it is probably fair to say that India can expect another excellent year of growth and income generation. Overview of Consumer Finance and Lending Structure and developments Lending and borrowing money are critical to a country\u2019s financial health. Banks traditionally played the role of prime lenders in India, disbursing loans for both commercial and personal purposes. However, extensive due diligence and long processes resulted in individuals turning to the unorganised sector or traditional moneylenders who charged exorbitant interest rates. In recent years, however, the lending market in India has developed into a dynamic playfield of various non-banking financial companies (NBFC), fintech enterprises and digital lending platforms. India\u2019s NBFCs continue to be instrumental in driving credit growth for the country\u2019s underserved segments. In the last few years, the contribution of NBFCs to India\u2019s gross domestic product (GDP) was well above 10% and these entities now account for more than 25% of credit, thus NBFCs role in driving consumption, investment, and financial inclusion make them a vital component of the country's economic engine. NBFCs are critical to India's financial ecosystem, particularly in underserved segment such as rural and semi-urban areas where banks have limited reach. Their importance lies in: \u2022 Financial inclusion: By providing credit to underserved regions. \u2022 Faster services: With simplified processes and doorstep delivery. \u2022 Priority sector lending: Addressing credit needs in agriculture, microfinance, and other unorganised sectors. \u2022 Economic growth: Supporting sectors like housing, infrastructure and small enterprises through financing. Outlook NBFCs remain a cornerstone of India\u2019s financial system, particularly to promote financial inclusion and economic growth. They can address the credit needs of underserved segments, leverage new sources of funds and embrace technological advancements. By integrating these strategies, NBFCs play a pivotal role in driving universal and sustainable economic growth, and ensure that the benefits of financial inclusion and technological innovation are widely shared across the nation. Chart A: India\u2019s rapid digiti sation journe y Ayushman Bharat Digital Mission Source: NPCI | ABDM | Straits Research | Inc42.com | Deloitte | Goldman Sachs | BCG | UIDAIIndia is digitising rapidly Rising income India stack powered the financial inclusion 1.4 billion Aadhaar generated 18.3 billion monthly transactions - March 2025 Investment in technology to reach USD 500 billion by 2030 Fintech market to reach USD 422 billion by 2029-CAGR of 27% during 2022-30 ONDC facilitated more than 200 million transactions till date Fintech $1 trillion in AUM & $200 billion in revenue by 2030 Insurance & insurtech $88 billion in size by 2030 77 crore+ ABHA ID's 53 crore+ Health Linked Records 391K+ Verified HFR ` K 34 The India digitisation story as depicted in Chart A, highlights the growing opportunity for the financial services businesses in India. Risks and concerns \u2022 Higher risk weights: Increased risk weights for loans from banks to NBFCs, making bank borrowing more expensive. However, effective 1 April 2025, the RBI has rolled back the higher risk weights previously assigned, and restored these to the pre-November 2023 levels. \u2022 Funding constraints: Smaller NBFCs with lower credit ratings face a fund crunch due to rising borrowing costs and limited financing \/ refinancing options. \u2022 Shallow bond market: India\u2019s debt market lacks depth and liquidity, limiting access to diversified domestic funding. \u2022 Cost pressures: Rising credit costs may affect NBFCs\u2019 profitability. \u2022 Overseas borrowing challenges: While attractive due to reduced hedging costs, overseas funding is still at a nascent stage for many NBFCs. Overview of the Insurance Sector Structure and developments The Indian insurance market is the tenth largest in the world. There are 60 insurers registered with the Insurance and Regulatory Development Authority of India (IRDAI), comprising 26 life- and 34 non-life insurers. The industry continues to be underpenetrated compared to most developed and developing economies, with a penetration less than 4% (ratio of total insurance premiums to the country\u2019s nominal GDP). For the insurance sector, FY2025 saw significant regulatory changes. On the one hand, these changes were aimed at bringing more transparency, efficiency, enhancing governance standards, streamlining regulations and prioritising policyholder interests. On the other, these contributed to muted premium growth for both life and non-life insurers especially during the second half of the year when most of these changes came in play. During the year, existing regulations were consolidated into few ones, minimising the need for regulatory approvals and enhancing operational efficiency. The Board\u2019s involvement in regulatory matters has been heightened. Governance mechanisms have been fortified to uphold transparency and accountability. Crucial steps have been taken to safeguard policyholder interests by revising turnaround times with respect to various policy services, to simplify claim processes and to ensure timely claim settlements. These have been major steps in transforming the regulatory framework from a \u2018rule-based\u2019 to a \u2018principle-based\u2019 approach. Despite operational challenges, there is now an enhanced focus on expanding insurance coverage to underserved areas by tagging rural obligations to the gram panchayat level; and the motor third party obligations require insurers to cover uninsured vehicles, so as to increase the number of insured vehicles. These reforms are expected to foster a more competitive and innovative insurance market, ultimately benefiting consumers through improved service delivery, enhanced product offerings and greater accessibility. These were followed by proposed amendments to existing insurance laws to support 100% foreign direct investment (FDI), composite insurance and to insurers offering value added services, to name a few. The IRDAI has continued in its path of positive regulatory developments with an aim to attain its vision of \u2018Insurance for All\u2019 by 2047. The second half of the year was more eventful with two regulations negatively impacting growth in short term. Life insurers were hit by the new surrender value regulations which assured higher surrender values to the customers which negatively impacted margins for insurers. This required the life insurers to modify their product structures as well as commercial terms with all distributors. Non-life insurers were hit by the change in accounting for long-term products, which required amortisation of long-term premium over the policy duration. Both these regulations impacted growth in H2 FY2025. The industry continued its work on setting up the electronic insurance marketplace (Bima Sugam), a comprehensive life, general and health insurance product for the lower income segments (Bima Vistar) and a new form of intermediary for the rural market (Bima Vaahak). On the investment front, FY2025 started on a positive note, as the Indian stock market indices scaled new lifetime highs. Stability in interest rates coupled with relatively moderate inflation led to excellent returns on equity and debt investments in H1 FY2025. However, these gains were short-lived. H2 FY2025 saw large selloff on account of stretched valuations, weak earnings, heavy foreign capital outflow amid rising US bond yields and the dollar, as well as global uncertainty arising from geopolitical tensions and escalation of US tariffs. The general insurance sector was also impacted due to lower sale of new vehicles, limited volume growth in retail health policies and fall in premium rates in the fire business. Increasing incidence of losses from natural catastrophes ensured that claims for the general insurance sector remained elevated. Outlook The Indian economy continues to witness strong fundamentals across sectors and will remain the fastest growing in the world. Such an economy backed by a favourable regulatory environment should drive insurance market growth in India. The insurance sector in India is hence expected to be among the fastest growing versus other economies. Lower insurance penetration and density (as depicted in Chart B), rising income levels, larger proportion of earning population, manufacturing growth and rapid infra development provide considerable scope for insurance and reinsurance in India in the immediate and near future. China South Korea Thailand Taiwan Singapore Japan Malaysia India 1.8 6.01.9 3.21.8 2.11.41.0Non-life insurance penetration as a % of GDP China South Korea Thailand Taiwan Singapore Japan Malaysia India 2.1 5.03.4 7.17.4 6.83.72.8Life insurance penetration as a % of GDP China South Korea Thailand Taiwan Singapore Japan Malaysia India 274 1,635 244 2,285 6,264 2,245 425 Life insurance density (US$) 70 China South Korea Thailand Taiwan Singapore Japan Malaysia India 234 1,968 140 1,022 1,536 693 165 Non-life insurance density (US$) 25 * Insurance penetration: total insurance premium\/ gross domestic product (GDP) Insurance density: average amount of insurance premium paid per person Chart B: Insurance penetration and density* Risks and concerns Key risks for the sector include (a) growing number and impact of natural calamities due to global warming; (b) data and cyber security risk arising from high level of digitisation in the industry; (c) market risk arising from growing geopolitical tensions and trade wars; (d) technology disruptions; and (e) enhanced competition through new entrants with the sector opening up (with proposals for 100% FDI and lower capital requirements). Consumer Finance and Lending Bajaj Finance Ltd. (BFL) BFL is one of India\u2019s largest non-banking financial companies (NBFCs). It is a customer-centric, digital-first enterprise with omnipresence across physical, mobile and web channels led by various payment platforms that combine different payment instruments. Focusing on India\u2019s mass affluent customers and above in both urban and rural, it uses a strategy of cross-selling with the smart use of state-of-the-art data, innovations and analytics to transform customer experience and create growth opportunities. It offers diversified financial services and optimises the mix of risk so as to regularly generate significant and sustainable profits. Business update BFL is present in 4,263 locations across India, including 2,681 in rural\/smaller towns and villages. Geographical expansion, large customer franchise and rapid adoption of digital technology continue to be critical pillars of Bajaj Finance\u2019s growth. It focuses on 10 broad categories. These are: (i) consumer lending (sales finance); (ii) personal loans; (iii) SME lending; (iv) auto financing; (v) rural lending; (vi) gold loans; (vii) commercial lending; (viii) loan against securities; (ix) deposits; and (x) partnerships and services. In FY2025: \u2022 BFL operated through over 232,000 distribution points across India. \u2022 It acquired a record 18.18 million new customers, taking its existing customer franchise to 101.82 million as on 31 March 2025, or a growth of 22% over 31 March 2024.BFL: Consolidated Performance Highlights for FY2025 \u2022 Customer franchise grew by 22% to 101.82 million. \u2022 Number of new loans booked increased by 20% to 43.42 million. \u2022 Number of customers on Bajaj Finserv App was 70.57 million. \u2022 Assets under management (AUM) increased by 26% to I 416,661 crore. \u2022 Net total income (NTI) increased by 24% to I 44,954 crore. \u2022 Pre-provisioning operating profit grew by 25% to I 30,028 crore. \u2022 Profit after tax (PAT) grew by 15% to I 16,638 crore. \u2022 Return on average assets (RoA) of 4.6% and return on average equity of 19.2%. \u2022 Capital adequacy ratio as on 31 March 2025 was 21.93%, Tier-I adequacy was 21.09% including a common equity tier 1 (CET1) ratio of 20.77% which are well above the RBI norms \u2014 making it one of the best capitalised large NBFCs in India. BFL101.82 million CUSTOMER FRANCHISEK 416,661 crore AUM (\u2191 22% YoY) (\u2191 26% YoY) FY2021 FY2020FY2022FY2023FY2024FY2025 152,947 147,153197,452247,379330,615416,661Chart C: BFL\u2019s Consolidated assets under management (L In Crore)Table 1 gives a snapshot of BFL\u2019s AUM across its different businesses. Chart C depicts BFL\u2019s consolidated AUM over the years. Table 1: BFL \u2019s assets under management (` In Crore) ParticularsStandalone Consolidated FY2025 FY2024 Change FY2025 FY2024 Change Two and three-wheeler finance 17,319 19,742 (12%) 17,319 19,742 (12%) Urban sales finance 29,109 23,448 24% 29,109 23,448 24% Urban B2C 86,096 64,471 34% 87,696 66,093 33% Rural sales finance (excluding gold loan) 7,944 6,209 28% 7,944 6,209 28% Rural B2C 21,467 17,607 22% 21,467 17,607 22% SME lending (excluding car loan) 50,230 38,498 30% 50,345 38,470 31% Loans against securities 20,872 18,221 15% 25,377 22,038 15% Commercial lending 28,285 22,006 29% 27,760 22,006 26% Mortgages 27,327 22,938 19% 129,461 103,316 25% Gold loans 8,307 4,599 81% 8,307 4,599 81% Car loans 11,876 7,087 68% 11,876 7,087 68% Total 308,832 244,826 26% 416,661 330,615 26% 5 year CAGR is 23% Table 2 gives BFL\u2019s standalone and consolidated financial performance; and Table 3 highlights the key ratios. Table 2: BFL\u2019s financial performance (` In Crore) ParticularsStandalone Consolidated FY2025 FY2024 Change FY2025 FY2024 Change Interest income 51,549 40,783 26% 61,164 48,307 27% Interest expenses 18,437 13,843 33% 24,771 18,725 32% Net interest income (NII) 33,112 26,940 23% 36,393 29,582 23% Other operating income 7,871 6,163 28% 8,561 6,676 28% Net total income (NTI) 40,983 33,103 24% 44,954 36,258 24% Operating expenses 13,968 11,478 22% 14,926 12,325 21% Pre-provisioning operating profit27,015 21,625 25% 30,028 23,933 25% Loan losses and provisions 7,883 4,572 72% 7,966 4,631 72% Share of profit from associates - - - 18 8 125% Exceptional items#2,544 - - - - - Profit before tax (PBT) 21,676 17,053 27% 22,080 19,310 14% Profit after tax (PAT) 16,662 12,644 32% 16,779 14,451 16% Profit after tax (PAT) attributable to Owners of the Company16,662 12,644 32% 16,638 14,451 15% Earnings per share (EPS) basic, in I269.33 207.27 30% 268.94 236.89 14% Earnings per share (EPS) diluted, in I268.58 206.47 30% 268.20 235.98 14% Book value per share, in I 1,420.93 1,168.31 22% 1,560.89 1,243.99 25% # On 13 September 2024, the Company has sold 428,571,428 equity shares of BHFL at ` 70 each, aggregating to ` 3,000 crore, as part of BHFL's Initial Public Offer. This has resulted in a gain of ` 2,544 crore (net of issue expenses). Table 3: BFL\u2019s key ratios on consolidated basis Ratios FY2025 FY2024 Net interest income (NII) to average loans receivable 9.91% 10.41% Total operating expenses to NTI 33.20% 33.99% Return on average loans receivable (ROA) 4.57% 5.08% Return on average equity (ROE) 19.19% 22.05% Capital to risk-weighted assets ratio (CRAR) * 21.93% 22.52% Tier I * 21.09% 21.51% Tier II * 0.84% 1.01% Gross NPA 0.96% 0.85% Net NPA 0.44% 0.37% Provisioning coverage ratio (PCR) 54% 57% * These ratios are on standalone basis. Bajaj Housing Finance Ltd. (BHFL) Ranked among the largest housing finance companies in India, Bajaj Housing Finance Ltd. (BHFL), a listed subsidiary of BFL, is registered as a non-deposit taking housing finance company with the National Housing Bank (NHB) since September 2015 to carry the business of housing finance. BHFL started its lending operations from July 2017. BHFL is regulated by the RBI and supervised by the NHB. BHFL crossed two significant milestones during FY2025: \u2022 In its eighth year of operations, it comfortably surpassed the AUM milestone of I 100,000 crore. \u2022 It got listed on the equity segment of the National Stock Exchange and the Stock Exchange, Mumbai (BSE) with its IPO in September 2024 \u2014 of I 6,560 crore, of which the primary issue was I 3,560 crore. The Company offers full mortgage product suite spanning: (i) home loans; (ii) loans against property; (iii) lease rental discounting; (iv) developer financing; and (v) others, covering non-collateralised loans. Through this varied product suite, BHFL addresses the diverse mortgage needs of all retail customer segments, which include salaried individuals and self-employed professionals. It serves all sub-segments, including prime, near-prime, and affordable customers; and offers various transaction types such as purchase, resale, self-construction and balance transfer. Through its commercial product suite, which includes construction finance and lease rental discounting, BHFL caters to a wide range of marquee clients, including real estate developers, high net worth individuals (HNIs), real estate investment trusts (REITs), sovereign wealth funds and corporations. BHFL holds the highest credit rating of AAA\/stable from both CRISIL and India Ratings for its long-term debt programme; and A1+ from CRISIL and India Ratings for its short-term debt programme. These ratings are supported by strong parent backing and reflect the Company's stable financial position and repayment capability. BHFL: Performance Highlights for FY2025 \u2022 Assets under management (AUM): increased by 26% to I 114,684 crore. \u2022 Net total income (NTI): rose by 23% to I 3,597 crore. \u2022 Operating expenses (Opex): increased by 6% to I 747 crore. \u2022 The ratio of Opex to NTI improved to 20.8% for FY2025 from 24.0% in FY2024. \u2022 Pre-provisioning operating profit grew by 28% to I 2,850 crore. \u2022 Profit after tax (PAT) rose by 25% to I 2,163 crore. \u2022 Capital adequacy ratio stood at 28.24% as of 31 March 2025. The Tier-1 ratio was 27.72%; both well above regulatory norms.A snapshot of BHFL\u2019s performance in FY2025 is given below. BHFLK 114,684 crore AUM K 2,163 crore PAT (\u2191 26% YoY) (\u2191 25% YoY) Table 4 gives BHFL\u2019s assets under management(AUM) and Table 5 gives its standalone financial performance. Table 4: BHFL\u2019s AUM across major business verticals (` In Crore) Particulars FY2025 FY2024 Change Housing loans (including top ups) 64,447 52,819 22% Loan against property 12,262 9,568 28% Lease rental discounting 21,913 17,637 24% Developer finance 14,346 9,599 49% Other loans 1,716 1,747 (2%) Total 114,684 91,370 26% Table 5: BHFL\u2019s standalone financials (` In Crore) Particulars FY2025 FY2024 Change Interest income 8,986 7,202 25% Interest expenses 5,979 4,692 27% Net interest income (NII) 3,007 2,510 20% Other operating income 590 415 42% Net total income (NTI) 3,597 2,925 23% Operating expenses 747 703 6% Pre-provisioning operating profit 2,850 2,222 28% Loan losses and provisions 80 61 31% Profit before tax (PBT) 2,770 2,161 28% Profit after tax (PAT) 2,163 1,731 25% Earnings per share (EPS) basic, in I 2.67 2.58 3% Bajaj Financial Securities Ltd. or Bajaj Broking Bajaj Financial Securities Ltd. (\u2018BFSL\u2019, \u2018BFinsec\u2019, \u2018Bajaj Broking\u2019) functions as a wholly-owned subsidiary of BFL. It is registered with the SEBI, both as a stockbroker and as a depository participant. In FY2025, Bajaj Broking significantly broadened its reach to 46 locations and 48 branches, providing a comprehensive suite of services to over 979,000 clients in retail and HNI, encompassing DP services, Broking, Margin Trading Facility, and Financing for Offers For Sale. The performance highlights of Bajaj Broking for FY2025 are given below: \u2022 Margin Trade Financing (MTF) book increased by 18% to I 4,505 crore. \u2022 Net interest income (NII) increased by 122% to I 249 crore. \u2022 Net total income (NTI) increased by 58% to I 441 crore. \u2022 Profit after tax (PAT) increased by 148% to I 139 crore. Bajaj BrokingOver 979,000 CUSTOMER FRANCHISE K 139 crore PAT (\u2191 40% YoY) (\u2191 148% YoY) Associate Companies of Bajaj Finance Ltd. There are two associate companies of BFL. These are: a) Snapwork Technologies Pvt. Ltd. which specialises in delivering innovative digital solutions, and where BFL holds a stake of 41.50% on a fully diluted basis; and b) Pennant Technologies Pvt. Ltd. which provides end-to-end lending solutions through its pennApps Lending Factory (PLF) \u2014 a comprehensive, highly configurable and scalable lending platform that streamlines the entire loan lifecycle. Here BFL holds a stake of 26.53% on a fully diluted basis. Protection and Savings Status of Joint Venture with Allianz SE BFS has executed Share Purchase Agreements (SPAs) for the acquisition of 26% equity stake owned by Allianz in its insurance subsidiaries, viz. Bajaj Allianz General Insurance Company Ltd. (BAGIC) and Bajaj Allianz Life Insurance Company Ltd. (BALIC), with participation by the promoter and promoter group entities of BFS. The agreed consideration for 26% stake in BAGIC and BALIC is I 13,780 crore and I 10,400 crore, respectively, aggregating I 24,180 crore. The acquisition will be in one or more tranches, of which the first tranche shall be at least 6.1% stake. Upon completion of the first tranche, the joint venture agreements between the Company and Allianz SE will be terminated. Pursuant to the above, BFS would be acquiring from Allianz, subject to approvals of the Competition Commission of India, Insurance Regulatory Development Authority of India and other customary approvals, 1.01% equity stake in each of the companies as a part of the first tranche of acquisition. BFS has also executed Share Purchase Agreement (SPA) for the acquisition, subject to required regulatory approvals, of 50% equity stake owned by Allianz in Bajaj Allianz Financial Distributors Ltd. (BAFDL), a joint venture in which the Company owns 50% of the paid-up equity share capital. Once the joint ventures are terminated under the terms of the SPA, the Bajaj group and Allianz aim to independently pursue their insurance strategies in India. The acquisitions mark a significant milestone. These bring the two insurance ventures entirely into the fold of the Bajaj group, one of India\u2019s most respected brands. Allianz and Bajaj are fully committed in ensuring a smooth transition such that interests of policyholders, intermediaries and other stakeholders are least affected. BFS and Allianz have also entered into agreements that will ensure continuity of reinsurance and other services during the transition. BAJAJ GROUP TO ACQUIRE ALLIANZ's 26% STAKE in BAGIC and BALIC AS PER THE SPAsK 24,180 crore AGREED CONSIDERATION General Insurance: Bajaj Allianz General Insurance Company Ltd. (BAGIC) BAGIC is one of India\u2019s leading composite general insurers offering all types of general insurance products including motor, health as well as various forms of commercial lines of insurance such as fire, engineering, liabilities and marine. It also participates in the tender driven government health and crop insurance schemes run across various states. It has built a strong retail franchise and retained a leading position among insurers. Its business is based on the foundation of a quality product portfolio, supported by strong underwriting, multi-channel distribution and prudent financial management. In a market where many peers continue to chase market share, BAGIC has, since its inception, differentiated itself by focusing on a profitable and diversified portfolio, and by balancing growth with profitability. With a positioning of \u2018Caringly Yours\u2019, it remains one of the most respected brands in general insurance known for its customer service. Industry update The IRDAI has introduced a change in the accounting of premium for long-term products (i.e. where policy duration is of more than 12 months) through the Master Circular on \u2018Actuarial, Finance & Investment functions of Insurers\u2019 effective 1 October 2024. Accordingly, premium on all long-term policies is to be recognised over the contract duration (on 1\/n basis where \u2018n\u2019 is the policy duration), which implies that only first year premium will be recognised as gross written premium (GWP) in the year of receipt and the balance collection will be reflected as Advance Premium to be accounted as GWP in the subsequent years. This defers premium recognition and negatively impacts the Combined Ratios (COR) for the year. Given these changes, the figures for FY2025 are not comparable with prior years. It should be noted that the change is purely an accounting one; the economics of insurance contracts remain unaffected. The general insurance industry (private and public multi-line players) recorded gross direct premium income (GDPI) growth of 5.2% on 1\/n basis (old basis 7.0%) in FY2025. The data for the industry and BAGIC are shown in Table 6. Table 6: Gross direct premium in India \u2013 the general insurance industry* It may be noted that given the change in accounting for long-term products effective 1 October 2024, numbers may not be comparable with previous periods. (` In Crore) Gross direct premiumGross direct premium excluding tender driven government schemes Particulars FY2025 FY2024 FY2025 FY2024 BAGIC 21,417 20,473 16,090 14,910 Private sector 162,899 155,090 141,744 133,387 Public sector 95,221 90,252 86,706 80,622 Industry 258,120 245,343 228,450 214,010 Standalone health insurers (SAHI) 38,436 33,119 38,436 33,119 Industry including SAHI 296,555 278,462 266,886 247,129 Growth rates BAGIC 4.6% 33.5% 7.9% 20.4% Private sector 5.0% 17.5% 6.3% 16.4% Public sector 5.5% 9.0% 7.5% 8.5% Industry 5.2% 14.2% 6.7% 13.3% Standalone health insurers (SAHI) 16.1% 26.2% 16.1% 26.2% Industry including SAHI 6.5% 15.5% 8.0% 14.9% Source: GI Council figures, IRDA website and IRDA, Handbook of Insurance Statistics. * Excluding specialised insurers. Business update \u2022 BAGIC posted a gross direct premium income (GDPI) growth rate of 4.6% in FY2025, versus industry growth of 5.2%. The impact of the new regulation is higher for BAGIC than the industry due to a higher proportion of long-term business. Excluding the impact of change in regulation on long-term insurance contracts, growth for BAGIC is 7.3% versus industry growth of 7.0%. \u2022 During the year, BAGIC wrote premium of I 2,067 crore (PY I 2,795 crore) under the government crop insurance schemes and I 3,260 crore (PY I 2,767 crore) crore under the Ayushman Bharat Government Health Scheme, thus continuing its endeavour of financial inclusion. \u2022 Excluding the volatile tender-driven crop insurance and government health business, BAGIC posted a GDPI growth rate of 7.9%. Excluding the impact of the change in regulation on long-term insurance contracts, BAGIC\u2019s GDPI grew by 11.6% versus industry growth at 8.8%. \u2022 BAGIC continues to be among the most profitable general insurers in India. Its COR of 102.3% in FY2025 (COR on old accounting basis for long-term contracts would be 101.7%) is expected to be among the best in the industry, reflecting a sound balance between growth and profitability. BAGIC has achieved this by: (a) robust and prudent underwriting practices driven by strong analytics based on past claims experience; (b) diversified exposure across business lines and geographies; (c) granular loss ratio modelling - geography, distribution, car make\/model, vintage, etc.; (d) robust investigation and loss management; (e) generation of cash flows through strong retention of premium and judicious investments of the proceeds; and (f) focus on high quality customer service. \u2022 BAGIC has a diversified and among the largest multi-channel distribution network consisting of banks, NBFCs, individual agents, motor insurance service providers, point of sales persons, web-sales and its proprietary direct to customers\u2019 channel. It continued to expand the network of bancassurance partners including private banks, public sector banks, regional banks, small finance banks and cooperative banks. Indeed, BAGIC has one of the largest networks of bancassurance partners in general insurance. \u2022 BAGIC has also embarked upon journey to expand its presence in tier 3 and below cities via its Geo channel to take the general and health insurance business to semi-urban and rural parts of the country. It opened nine new offices during FY2025 and continues to open new offices across the country. \u2022 BAGIC continues to balance its product mix across retail and commercial lines depending on profit pools that it identifies from time to time. \u2022 BAGIC continued to invest in technology and push towards enhanced digital services with capabilities like AI enabled BOT (called BOING), which serviced more than 2.12 million unique customers with over 176 million conversations. Its customer facing mobile app, Caringly Yours, crossed 5.37 million downloads. Its innovative mobile app for the farming community, Farmitra , saw more than 1.62 million downloads. BAGIC has also undertaken investments in Gen AI solutions. \u2022 BAGIC\u2019s core policy administration system is being transformed to enhance agility. During the year, entire health (including group), crop insurance, surety bonds, motor third party insurance products went live in the system. BAGIC: Performance Highlights for FY2025 \u2022 Gross written premium (GWP) stood at I 21,583 crore in FY2025, or a growth of 5%. Excluding tender driven business of crop and government health insurance, BAGIC\u2019s growth was 8%. Excluding impact of new regulation on long-term insurance contracts, GWP growth was at 7% (excluding crop and government health at 12%). Impact of the regulation on GWP was I 551 crore during FY2025. \u2022 BAGIC\u2019s market share in the industry on GDPI basis, including standalone insurers, stood at 7.2% in FY2025. \u2022 Net earned premium was I 9,565 crore. \u2022 The combined ratio was 102.3%. (Combined ratio on old accounting basis was 101.7%). \u2022 Profit after tax for FY2025 was I 1,832 crore with a growth of 18%. \u2022 AUM was I 33,115 crore. \u2022 Highest solvency ratio among comparable industry players at 325%. \u2022 48 million policies were issued in the year, versus some 38 million in FY2024. \u2022 6.47 million claims were reported in the year, versus 5.97 million in FY2024. \u2022 BAGIC\u2019s grievance ratio continue to be among the lowest in the industry and its NPS scores continue to outperform the market, reflecting strong customer trust. Financial performance BAGIC\u2019s financial performance for FY2025 is summarised in Table 7. Chart D plots BAGIC\u2019s PAT over the last five years. Table 7: BAGIC\u2019s financial performance (` In Crore) Particulars FY2025 FY2024 % Change Gross written premium (GWP) 21,583 20,630 5% GWP excluding crop and government health 16,256 15,068 8% GWP excluding crop, government health & 1\/n impact of long -term products16,807 15,068 12% Net earned premium 9,565 8,884 8% Underwriting result (77) (166) Profit before tax 2,445 2,055 19% Profit after tax 1,832 1,550 18% Claims ratio 74.6% 73.8% Combined ratio* 102.3% 99.9% Combined ratio (excluding impact of new regulations)* 101.7% 99.9% Return on average equity ^ 16.0% 15.2% * Combined ratio is calculated according to the Master Circular on \u2018Preparation of Financial Statements of General Insurance Business\u2019 issued by the IRDAI effective from 1 April 2013. ^ Excluding the surplus capital (at 200% solvency) the return on average Equity was 22.4%BAGIC 48 million POLICIES ISSUED K 1,832 crore PAT (\u2191 18% YoY) FY2021 FY2020FY2022FY2023FY2024FY2025 1,330 9991,339 1,348 1,550 1,832 Chart D: BAGIC's profit after tax (L In Crore) Cash and investments \u2022 BAGIC\u2019s cash and investments as on 31 March 2025 were I 33,115 crore, versus I 31,196 crore in the previous year. Cash flow generation continued to be strong. Capital and solvency \u2022 Paid-up capital, including share premium, stood at I 277 crore as at 31 March 2025. No fresh capital was infused in the year. \u2022 BAGIC continues to be one of the most efficient users of capital in the private sector \u2014 as measured by the ratio of GWP to share capital and by GWP to shareholders\u2019 equity. \u2022 Shareholders\u2019 equity of BAGIC was I 12,498 crore as on 31 March 2025, versus I 11,474 crore a year earlier. \u2022 As on 31 March 2025, BAGIC\u2019s solvency margin was at 325%, which is well above the normal regulatory requirement of 150%. Life Insurance, Savings and Retirement: Bajaj Allianz Life Insurance Company Ltd. (BALIC) With a brand positioning of getting Life Goals Done, BALIC is committed to offer value-packed and innovative products which are simple to understand and easy to buy. These are designed to meet long-term life goals of individual and group customers, ranging from protection, wealth creation to retirement solutions and more; and are backed by a well-diversified and strong multi-channel distribution network, data analytics, tech-enabled services and service touchpoints to ensure a simplified and personalised insurance experience with a \u2018Customer First\u2019 approach. Over the last seven to eight years, BALIC has achieved significant growth and market share. As part of its journey towards profitable and sustainable growth, BALIC initiated a change in strategy which coincided with the new product regulations issued by the IRDAI, and resulted in lower growths in H2 FY2025. This new strategy aims to leverage the scale already achieved for further expansion of its margins. Industry update The life insurance industry recorded a growth of 10.5% in individual rated new business (IRNB) premium in FY2025 over FY2024 after changes introduced in the surrender value regulations which came into effect from 1 October 2024. Within the industry, private sector grew by 15.1%, while LIC grew by 0.7%. BALIC recorded growth in IRNB of 11.7% for FY2025. Par, retail term and ULIP product basket were the key growth drivers. Table 8 gives the data.5 year CAGR is 13% Table 8: New business premium in India \u2013 the life insurance industry (` In Crore) Particulars Individual rated new businessNew business (NB) FY2025 FY2024 FY2025 FY2024 BALIC 7,067 6,326 12,293 11,494 Private sector 85,023 73,871 170,667 155,437 LIC 35,352 35,104 226,670 222,523 Industry 120,376 108,975 397,337 377,960 Growth rates BALIC 11.7% 21.3% 7.0% 7.0% Private sector 15.1% 8.0% 9.8% 12.1% LIC 0.7% (1.3%) 1.9% (4.0%) Industry 10.5% 4.8% 5.1% 2.0% Market share Private industry 70.6% 67.8% 43.0% 41.1% LIC 29.4% 32.2% 57.0% 58.9% Source: Life Council statistics . Note: Individual rated new business premium = (100% of first year premium and 10% of single premium excluding group products). Business update BALIC recorded gross written premium (GWP) of I 27,160 crore and an AUM of I 123,734 crore in FY2025. In terms of individual rated new business (IRNB) premium, a key focus area, BALIC registered a growth of 11.7%, while the industry grew at 10.5%. BALIC\u2019s growth is evident from the Chart E below, with a five year (FY2020-FY2025) compounded annual growth rate (CAGR) of 30%. FY2021 FY2020FY2022FY2023FY2024FY2025 2,468 1,927 3,686 5,214 6,326 7,067 Chart E: BALIC's individual rate d new business (IRN B) premium (L In Crore) \u2022 BALIC registered a 10 basis points increase in market share from 5.8% in FY2024 to 5.9% in FY2025 on IRNB basis. Within the private industry, its market share de-grew by 30 basis points from 8.6% in FY2024 to 8.3% in FY2025. \u2022 BALIC continued with its emphasis on a balanced and sustainable product mix. As a result, in the product mix measured on IRNB basis, the proportion of participating business was 24% in FY2025 (versus 27% in 5 year CAGR is 30% BALIC: Performance Highlights for FY2025 \u2022 Individual rated new business (IRNB) premium grew by 12% to I 7,067 crore. \u2022 New business premium grew by 7% to I 12,293 crore. \u2022 Renewal premium grew by 29% to I 14,867 crore. \u2022 Gross written premium (GWP) increased by 18% to I 27,160 crore. \u2022 PAT was I 508 crore. \u2022 Value of New Business (VNB), a key metric used to measure profitability of life insurance businesses, increased by 9% to I 1,152 crore. \u2022 The embedded value of BALIC at FY2025 stood at I 23,805 crore up from I 21,726 crore at FY2024. \u2022 AUM was I 123,734 crore. \u2022 Highest solvency ratio in the industry: 359%. \u2022 Covered 16.5 million group lives in FY2025. \u2022 13th month persistency* was at 82% and 61st month persistency* was at 54%. \u2022 Retail Claims Settlement Ratio of 99.3% and Group Claims Settlement Ratio of 99.8% (FY2025). *Policies issued in the March to February period of the relevant years.FY2024); non-participating savings, annuity and protection products was 31% in FY2025 (34% in FY2024); and ULIP was 45% in FY2025 (39% in FY2024). Growth in ULIP products was due to buoyant equity markets. \u2022 BALIC has a robust risk management framework. Products carrying financial guarantees are hedged against interest rate and cash flow risk through forward rate agreements with reputed banks within the framework of a Board-approved policy. \u2022 Some other key achievements of FY2025 for BALIC are: \u2212An increase in persistency across majority cohorts, lower customer grievances and improvement in claims settlement ratios. \u2212Enhancement of product portfolio with launch of new products in various segments. \u2212Balancing product mix between traditional and ULIP as well as between individual and group business. The product mix is well diversified across par, non-par savings, protection, annuity and ULIPs. \u2212Continue to strengthen and build-up agency via an \u2018Add and Grow\u2019 strategy through new initiatives with higher proportion of variable cost channels to drive efficiencies. This has led to the BALIC building one of the largest agency channels in the private life insurance space with over 161,000 agents. \u2212BALIC Direct which involves selling by BALIC\u2019s own staff increased its presence to 407 cities, with dedicated verticals for various customer segments such as Service to Sales, New to BALIC (NTB) and Defense. \u2212Continued focus on reducing concentration risk in institutional business with addition of new bancassurance partners and developing new relations with quality partners in non-banking sector. \u2212Customer First Programme: Increase in number of customers opting for auto-pay helped focusing on faster issuances, claim settlement and driving first time right (FTR). Introduced a revamped customer app and customer communication protocol for a hyper-personalised experience and seamless policy servicing. \u2212Strong digital adoption with increase in unique customer logged in digital assets and customer transactions. BALIC K 123,734 crore AUM 99.3% CLAIMS SETTLEMENT RATIO (RETAIL CLAIMS) (\u2191 13% YoY) Financial performance BALIC\u2019s financial performance for FY2025 is summarised in Table 9. Table 9: BALIC\u2019s financial performance (` In Crore) Particulars FY2025 FY2024 % Change Gross written premium (GWP) 27,160 23,043 18% New business premium 12,293 11,494 7% - Individual rated new business premium 7,067 6,326 12% - Group protection new business premium 2,616 2,315 13% Renewal premium 14,867 11,549 29% Profit After Tax (PAT) 508 563 (10%) - Policyholders\u2019 surplus (420) (247) - - Shareholders\u2019 profit after tax 928 810 15% Value of New Business (VNB) 1,152 1,061 9% Investments As on 31 March 2025, BALIC\u2019s AUM was I 123,734 crore, up by 13% from I 109,829 crore on 31 March 2024. Table 10 provides the data. Table 10: BALIC\u2019s assets under management (` In Crore) Particulars FY2025 FY2024 Shareholders\u2019 funds 10,289 10,993 Policyholders\u2019 linked funds 47,261 43,744 Policyholders\u2019 non linked funds 66,184 55,092 Total 123,734 109,829 Capital and solvency \u2022 Paid up capital, including share premium, stood at I 1,211 crore as at 31 March 2025. No fresh capital was infused in FY2025. \u2022 Including accumulated profit of I 9,459 crore, the shareholders\u2019 net worth was I 10,899 crore (FY2025). For the previous year, accumulated profits were I 9,449 crore, and shareholders\u2019 net worth was I 11,139 crore. \u2022 BALIC has a strong solvency ratio of 359% as on 31 March 2025, which is well in excess of the minimum regulatory requirement of 150%. Emerging Businesses Bajaj Finserv Direct Ltd. (BFSD) Bajaj Finserv Direct is an open architecture based diversified marketplace for financial services which attracts a large number of new-to-Finserv consumers, creates awareness and discovery of the Finserv brand and cross-sells products by leveraging technology and analytics. It also enhances the digital footprint of Bajaj Finserv through the offering of select digital technology services. Having started its journey in July 2018, it operates under two divisions, as explained below. BFSI Marketplace (Bajaj Markets) \u2022 It has two customer facing digital assets: (i) the \u2018Bajaj Markets Website\u2019 \u2013 which attracts some 237 million annual visits, and (ii) the \u2018Bajaj Markets mobile app\u2019 which is available on Android and iOS with total installs of some 32 million. Bajaj Markets operates under the concept of web is equal to app. \u2022 It partners with some of the leading companies in the BFSI sector. It has onboarded 96 partners, with 37 products ranging across retail banking (loans and credit cards), life insurance, general insurance, health insurance and investments (fixed deposits, mutual funds, national pension scheme etc.). \u2022 Open architecture offers a wide choice to customers. It has 55 lending partners including BFL. \u2022 It is a corporate agent for distribution of insurance products; and has 19 partnerships across general, life and health insurance, including BAGIC and BALIC. \u2022 It has integrated with ONDC through technology partnership and has started to offer electronics items. There are plans to foray into other similar fast moving consumer categories. Bajaj Technology Services \u2022 This focuses on building digital-first solutions for BFSI businesses, and reimagining customer experiences and operations by focusing on eight practices: (i) Digital Experience, (ii) Commerce, (iii) Data Engineering and Analytics, (iv) AI, (v) Cloud, (vi) CRM, (vii) Digital Agency and (viii) Engineering. \u2022 Bajaj Finserv and its subsidiaries have demonstrated the use of relevant technology at scale across various segments of financial services like lending and insurance. Capitalising on this and the capabilities gained while building our own marketplace, the division seeks to offer technology-based services to prospective clients in the financial services ecosystem. \u2022 Working with some of the leading global platforms like SalesForce, Adobe, Microsoft Azure and Amazon Web Services, Bajaj Technology Services helps customers in their digital transformation journey. It has developed real-world GenAI applications and AI accelerators designed to reduce time-to-market and maximise value of AI-powered solutions. \u2022 Bajaj Technology Services plans to drive business through local clients, GCCs set up in India and international clients (largely in the Middle East and the United States). Bajaj Markets recorded a revenue of I 598 crore and a loss of I 49 crore during FY2025 ( I 475 crore and I 73 crore respectively in FY2024). BFS and BFL have so far infused capital of I 808 crore in this venture. However, no further capital was infused during the year. The Company has turned cash positive in FY2025. BFSD237 million ANNUAL VISITS ON BAJAJ MARKETS WEBSITE K 598 crore REVENUE Bajaj Finserv Health Ltd. (Bajaj Finserv Health) Bajaj Finserv Health is health-tech venture that aims to transform the healthcare sector in India. It offers a range of healthcare solutions and services. The mission is to provide platforms for customers to manage their healthcare needs 360\u00b0, covering wellness, outpatient (OPD) and inpatient (IPD) services. Through an efficient digital platform, Bajaj Finserv Health seeks to: a) connect providers of healthcare such as hospitals, doctors, labs, etc. with consumers of healthcare, and b) provide healthcare and claims management services to payers of healthcare expenditure such as insurance companies, employers and government. In April 2024, Bajaj Finserv Health completed the acquisition of 100% stake in Vidal Healthcare Services Pvt. Ltd. (VHC) at an enterprise value of I 310.70 crore. VHC has a subsidiary, Vidal Health Insurance TPA Pvt. Ltd. which is registered as a Third-Party Administrator (TPA) with the IRDAI and has vast experience in healthcare administration in India. The VHC acquisition significantly augments capabilities of the company in the healthcare space. On a consolidated basis with Vidal, Bajaj Finserv Health has built a network of over 100,000 doctors, more than 5,000 diagnostic points and over 15,000 hospitals where outpatient (OP) transactions can be done on cashless basis.
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-984556358.py, line 11)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [19]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 198581, 325251, 526702, 843675, 1395076, 2175761, 3495507, 5773906, 9660250,


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [20]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [21]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The tall tower in the capital of France is a magnificent structure that stands proud amidst the city's busy streets. Its height is impressive, making it a striking landmark for locals and visitors alike. The tower is made of gleaming white stone that seems to reflect the vibrant hues of the Parisian sun. Atop the tower, there is a majestic clock tower, its elegant face etched with precise time markings that make it a beacon for the city's residents. The tower is surrounded by beautiful parks, which provide a serene haven for both tourists and the city's inhabitants. In the evening, when the city lights up, the tower transforms into


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [22]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [23]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [24]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
